In [1]:
import os 

import yfinance as yf 
import backtrader as bt 
from datetime import datetime

In [2]:
if not os.path.exists('yfdata'):
    os.mkdir('yfdata')

ticker = 'AAPL'
data = yf.download(ticker, period='1y', interval='1d')
data_path = os.path.join('yfdata', ticker+'.csv')
data.to_csv(data_path)

[*********************100%***********************]  1 of 1 completed


In [3]:
# SMA crossover
# price > SMA --> buy
# price < SMA --> sell
# NO short selling
# holding only 1-share

class SmaStrategy(bt.Strategy):
    params = (('ma_period', 20), )
    
    def __init__(self):
        self.data_close = self.datas[0].close
        self.order = None 
        self.price = None 
        self.comm = None
        self.sma = bt.ind.SMA(self.datas[0], period=self.params.ma_period)
        
    def log(self, txt):
        dt = self.datas[0].datetime.date(0).isoformat()
        print(f"{dt} | {txt}")
        
    def notify_order(self, order):
        self.order = None
        
    def next(self):
        if self.order:
            return
        
        if not self.position:
            if self.data_close[0] > self.sma[0]:
                self.order = self.buy()
                
        else:
            if self.data_close[0] < self.sma[0]:
                self.order = self.sell()
                
    def stop(self):
        self.log(f"(ma_period = {self.params.ma_period:.2f}) --- Terminal Value: {self.broker.getvalue():.2f}")

In [4]:
data = bt.feeds.YahooFinanceCSVData(dataname=data_path)

In [5]:
cerebro = bt.Cerebro(stdstats=False)
cerebro.adddata(data)
cerebro.optstrategy(SmaStrategy, ma_period=range(10, 31))
cerebro.broker.setcash(1000)
cerebro.run(maxcpus=4)

2022-02-18 | (ma_period = 12.00) --- Terminal Value: 1020.482022-02-18 | (ma_period = 11.00) --- Terminal Value: 1022.80

2022-02-18 | (ma_period = 13.00) --- Terminal Value: 1027.38
2022-02-18 | (ma_period = 10.00) --- Terminal Value: 1019.57
2022-02-18 | (ma_period = 14.00) --- Terminal Value: 1027.94
2022-02-18 | (ma_period = 15.00) --- Terminal Value: 1030.53
2022-02-18 | (ma_period = 17.00) --- Terminal Value: 1035.47
2022-02-18 | (ma_period = 18.00) --- Terminal Value: 1042.46
2022-02-18 | (ma_period = 16.00) --- Terminal Value: 1032.86
2022-02-18 | (ma_period = 19.00) --- Terminal Value: 1041.45
2022-02-18 | (ma_period = 20.00) --- Terminal Value: 1044.10
2022-02-18 | (ma_period = 21.00) --- Terminal Value: 1038.48
2022-02-18 | (ma_period = 22.00) --- Terminal Value: 1041.99
2022-02-18 | (ma_period = 23.00) --- Terminal Value: 1042.43
2022-02-18 | (ma_period = 24.00) --- Terminal Value: 1042.09
2022-02-18 | (ma_period = 26.00) --- Terminal Value: 1042.752022-02-18 | (ma_period =

[[<backtrader.cerebro.OptReturn at 0x7f63aea50610>],